# Atividade Prática 2.1 - Cobertura Florestal

* Disciplina _Inteligência Artificial Aplicada_
* Professora: Elloá B. Guedes (ebgcosta@uea.edu.br)
* Data de apresentação: 26 de janeiro de 2024
* Data limite de entrega: 01 de fevereiro de 2024


## Equipe
* Integrante 1: *Aurelio Aquino*
* Integrante 2: *Jailson Bina*
* Integrante 3: *Sthephany Costa*
* Integrante 4: *Erica Veras*
* Integrante 5: *Michelle de Carvalho*
* Integrante 6: *Fabiano Dolzanes*


## Contexto: Cobertura Florestal

Este conjunto de dados contém observações de árvores de quatro áreas da Floresta Nacional de Roosevelt, no Colorado. Todas as observações são variáveis cartográficas (sem sensoriamento remoto) de seções de floresta de 30 metros por 30 metros. Há mais de meio milhão de medições no total

## Base de Dados

Disponível em: https://www.kaggle.com/datasets/uciml/forest-cover-type-dataset

### Bibliotecas

Por hábito, a primeira célula do notebook costuma ser reservada para importação de bibliotecas.
A cada biblioteca nova acrescida, é necessário executar a célula para atualização e correta execução

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import ConfusionMatrixDisplay, accuracy_score, f1_score, precision_score, recall_score

### Abertura do Dataset

Abra o dataset e visualize o seu cabeçalho, isto é, os primeiros exemplos nele contidos.
Isto é útil para checar se a importação foi realizada de maneira adequada e se a disposição dos dados está de acordo para os próximos passos do trabalho.

In [ ]:
df = pd.read_csv('covtype2 - covtype2.csv')

### Análise Exploratória

1. Quantos exemplos há no dataset?
2. Quais os atributos existentes no dataset?
3. O atributo alvo é Cover_Type. A distribuição de classes no mesmo é uniforme?

In [ ]:
len(df)

In [ ]:
df.columns

In [ ]:
df['Cover_Type'].value_counts()

### Organização dos dados para treinamento

1. Remova os dados faltantes
2. Remova a coluna Cover_Type e atribua-a a uma variável Y
3. Atribua os demais valores do dataset a uma variável X
4. Efetue uma partição holdout 70/30 com o sklearn

In [ ]:
df = df.dropna()

In [ ]:
X = df.drop(columns=['Cover_Type'])

In [ ]:
Y = df['Cover_Type']

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3)

### Treinamento de uma RNA MLP para o problema

1. Treine uma RNA MLP Classificadora para este problema com uma única camada e 10 neurônios  
    1.1 Utilize a função de ativação ReLU  
    1.2 Utilize o solver Adam    
    1.3 Imprima o passo a passo do treinamento    
    1.4 Utilize o número máximo de épocas igual a 100 

In [ ]:
modelo = MLPClassifier(hidden_layer_sizes=(10,), activation='relu', solver='adam', verbose=True, max_iter=100)

In [ ]:
modelo.fit(X_train, Y_train)

## Aferição de Desempenho

2. Com o modelo em questão, após o treinamento, apresente:  
    2.1 Matriz de confusão para o conjunto de teste  
    2.2 Acurácia  
    2.3 F-Score Balanceado  
    2.4 Precisão  
    2.5 Revocação 

In [ ]:
Y_prev = modelo.predict(X_test)
ConfusionMatrixDisplay.from_predictions(Y_test, Y_prev)

In [ ]:
accuracy_score(Y_test, Y_prev)

In [ ]:
f1_score(Y_test, Y_prev, average='macro')

In [ ]:
precision_score(Y_test, Y_prev, average='macro')

In [ ]:
recall_score(Y_test, Y_prev, average='macro')

## Busca em Grade

Uma maneira padrão de escolher os parâmetros de um modelo de Machine Learning é por meio de uma busca em grade via força bruta. O algoritmo da busca em grade é dado como segue:

1. Escolha a métrica de desempenho que você deseja maximizar  
2. Escolha o algoritmo de Machine Learning (exemplo: redes neurais artificiais). Em seguida, defina os parâmetros ou hiperparâmetros deste tipo de modelo sobre os quais você dseja otimizar (número de épocas, taxa de aprendizado, etc.) e construa um array de valores a serem testados para cada parâmetro ou hiperparâmetro.  
3. Defina a grade de busca, a qual é dada como o produto cartesiano de cada parâmetro a ser testado. Por exemplo, para os arrays [50, 75, 100] e [10, 15], tem-se que a grade é [(50,10), (50,15), (75,10), (75,15), (100,10), (100,15)].
4. Para cada combinação de parâmetros a serem otimizados, utilize o conjunto de treinamento para realizar uma validação cruzada (holdout ou k-fold) e calcule a métrica de avaliação no conjunto de teste (ou conjuntos de teste)
5. Escolha a combinação de parâmetros que maximizam a métrica de avaliação. Este é o modelo otimizado.

Por que esta abordagem funciona? Porque a busca em grade efetua uma pesquisa extensiva sobre as possíveis combinações de valores para cada um dos parâmetros a serem ajustados. Para cada combinação, ela estima a performance do modelo em dados novos. Por fim, o modelo com melhor métrica de desempenho é escolhido. Tem-se então que este modelo é o que melhor pode vir a generalizar mediante dados nunca antes vistos.

Nesta busca em grande, contemple a utilização do objeto [GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html)

In [ ]:
neuron1 = [50, 75, 100]
neuron2 = [10, 15]

neurons = []
for i in neuron1:
  for j in neuron2:
    neurons.append((i,j))

neurons

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
parameters = {
    'hidden_layer_sizes': neurons,
    'activation': ['identity', 'logistic', 'tahn', 'relu'],
    'solver': ['adam', 'lbfgs', 'sqd']
}
# Utilizando k-fold padrão de 5 e aumentando número de tentativas para 20
# Total de 360 fits
searcher = GridSearchCV(MLPClassifier(verbose=True, n_iter_no_change=20), parameters, verbose=True)

In [ ]:
searcher.fit(X_train, Y_train)

In [ ]:
searcher.best_params_